# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet

#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
+ Formulas for the Bernoulli distribution: https://en.wikipedia.org/wiki/Bernoulli_distribution

In [10]:
import pandas as pd
import numpy as np
from scipy import stats

In [11]:
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')

In [12]:
# number of callbacks for black-sounding names
sum(data[data.race=='w'].call)

235.0

In [13]:
data.head()

,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,...,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
0,b,1,4,2,6,0,0,0,1,17,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
1,b,1,3,3,6,0,1,1,0,316,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
2,b,1,4,1,6,0,0,0,0,19,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
3,b,1,3,4,6,0,1,0,1,313,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
4,b,1,3,3,22,0,0,0,0,313,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Nonprofit


<div class="span5 alert alert-success">
<p>Your answers to Q1 and Q2 here</p>
</div>

In [14]:
w = data[data.race=='w']
b = data[data.race=='b']

In [15]:
# Your solution to Q3 here

<div class="span5 alert alert-success">
<p> Your answers to Q4 and Q5 here </p>
</div>

In [19]:
# Constants
ALPHA = 0.05

# 1. What test is appropriate for this problem? Does CLT apply?

In [16]:
n = len(data)
print(f'Sample size is {n}.')

Sample size is 4870.


As the population standard deviation is unknown and the question is about comparison between two samples, __two-sample t-test__ is appropriate to use here.

CLT should be applicable to the dataset for two reasons:
1. White-sounding and black-sounding names are assigned randomly to the resumes.
2. Sample size is large enough.

# 2. What are the null and alternate hypotheses?

P<sub>B</sub>: Probability of black-sounding names callback

P<sub>W</sub>: Probability of white-sounding names callback

H<sub>0</sub>: P<sub>B</sub> = P<sub>W</sub>

H<sub>1</sub>: P<sub>B</sub> != P<sub>W</sub>

# 3. Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches.

## Bootstrap Test

In [29]:
def perm_reps(data1, data2, func):
    """Concatenate & permute two data set and return replicate"""
    data_conc = np.concatenate((data1, data2))
    data_perm = np.random.permutation(data_conc)
    data_perm_1 = data_perm[:len(data1)]
    data_perm_2 = data_perm[len(data1):]
    rep = func(data_perm_1, data_perm_2)
    return rep


def draw_perm_reps(data1, data2, func, size=1):
    """Generate replicates from permutation"""
    reps = np.empty(size)
    for i in range(size):
        reps[i] = perm_reps(data1, data2, func)
    return reps


def prob_diff(data1, data2):
    """Calculate the probability difference"""
    prob_1 = np.sum(data1) / len(data1)
    prob_2 = np.sum(data2) / len(data2)
    return prob_2 - prob_1

def print_result(p_value, ALPHA=0.05):
    if p_value_bs > ALPHA:
        print('Fail to reject H0, race doesn\'t have significant impace on the rate of callback for resumes.')
    else:
        print('Reject H0, race does have significant impace on the rate of callback for resumes.')

In [25]:
prob_diff_obs = prob_diff(b.call, w.call)
prob_diff_reps = draw_perm_reps(b.call, w.call, prob_diff, size=10000)
p_value_bs = np.sum(prob_diff_reps >= prob_diff_obs) / len(prob_diff_reps)
print(f'p-value is {p_value_bs}')

p-value is 0.0


In [30]:
print_result(p_value_bs, ALPHA)

Reject H0, race does have significant impace on the rate of callback for resumes.


## Frequentist Test

In [31]:
n_b = len(b)
n_w = len(w)
p_b = np.sum(b.call) / n_b
p_w = np.sum(w.call) / n_w

p_hat = (n_b*p_b + n_w*p_w) / (n_b + n_w)
t = (p_b - p_w) / (np.sqrt(p_hat * (1 - p_hat) * (1/n_b + 1/n_w)))
p_value_t = stats.t.sf(np.abs(t), df=(n_b + n_w - 1))
print(f'p-value is {p_value_t}')

p-value is 2.0246589437951843e-05


In [32]:
print_result(p_value_t, ALPHA)

Reject H0, race does have significant impace on the rate of callback for resumes.


# 4. Write a story describing the statistical significance in the context or the original problem.

To examine racial discrimination in US job market. The idential resumes with randomly assigned black-sounding and white-sounding names are sent to employers.

Bootstrap and frequestist hypothesis tests are conducted on the callback rate of black-sounding and white-sounding names. Both tests suggest that race does have impact on the callback rate from employer.

# 5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

The analysis only suggests that race/name has impact on callback rate. It doesn't show that race/name is the most important factor. There may be other factors like education, experience, sex and city, which may have bigger impact on the callback. Further analysis on these factors needs to be done before making conclusion.